In [1]:
from imports import *
from functions import *
from auth import *
from constants import *

In [2]:
driver = create_driver()
wait = WebDriverWait(driver, 3)

In [3]:
login(driver, email, password)

⏳ Tentative de suppression du popup Fincaraiz...
🧨 Pop-up Fincaraiz supprimé.
Erreur lors de la connexion :
name 'wait' is not defined


In [ ]:
real_estate_types = ["apartamento", "apartaestudio", "cabana", "casa campestre", "casa lote", "finca", "habitacion"]


In [ ]:
select_area = driver.find_element(By.CSS_SELECTOR, ".ant-select-selector")
clear_ant_select_choices(driver)
select_area.click()

try:
    # Attendre que l'élément contenant "Casa" soit cliquable
    choices = driver.find_elements(By.CLASS_NAME, "ant-select-item-option-content")

    for choice in choices:
        if choice.text.lower() == "casa":
            choice_casa = choice
        if choice.text.lower() == "apartaestudio":
            choice_apartaestudio= choice
        if choice.text.lower() == "cabaña" or choice.text.lower() == "cabana":
            choice_cabana = choice
        if choice.text.lower() == "casa":
            choice_casa = choice
        if choice.text.lower() == "casa":
            choice_casa = choice
        if choice.text.lower() == "casa":
            choice_casa = choice
        if choice.text.lower() == "casa":
            choice_casa = choice
             
        
        print(choice.text)
        choice.click()

except Exception as e:
    print(f"choices not found:\n {str(e)}")

✅ Sélections supprimées.

Apartamento
Apartaestudio
Cabaña
Casa Campestre
Casa Lote
Finca
Habitación

choices not found:
 Message: element not interactable
  (Session info: chrome=135.0.7049.96)
Stacktrace:
	GetHandleVerifier [0x00007FF7779D5355+78597]
	GetHandleVerifier [0x00007FF7779D53B0+78688]
	(No symbol) [0x00007FF777788FDC]
	(No symbol) [0x00007FF7777E061D]
	(No symbol) [0x00007FF7777D2204]
	(No symbol) [0x00007FF7778070EA]
	(No symbol) [0x00007FF7777D1AB6]
	(No symbol) [0x00007FF777807300]
	(No symbol) [0x00007FF77782F2BB]
	(No symbol) [0x00007FF777806EC3]
	(No symbol) [0x00007FF7777D03F8]
	(No symbol) [0x00007FF7777D1163]
	GetHandleVerifier [0x00007FF777C7EF0D+2870973]
	GetHandleVerifier [0x00007FF777C796B8+2848360]
	GetHandleVerifier [0x00007FF777C96993+2967875]
	GetHandleVerifier [0x00007FF7779F019A+188746]
	GetHandleVerifier [0x00007FF7779F847F+222255]
	GetHandleVerifier [0x00007FF7779DD2D4+111236]
	GetHandleVerifier [0x00007FF7779DD482+111666]
	GetHandleVerifier [0x00007FF

In [ ]:
ant_selector = driver.find_element(By.XPATH, "//div[@class='ant-select-selector']")
ant_selector.click()

try:
    div_choices = driver.find_element(By.XPATH,"//div[@class='home-cover_content']")
    print(div_choices)
    for choice in
except Exception as e:
    print("div_choices not found")


try:
    # Attendre que les options soient visibles (s'il y a un délai de chargement)
    options = ant_selector.find_elements(By.CSS_SELECTOR, "div[class='ant-select-item-option-content']")
    # Afficher chaque option disponible
    for option in options:
        print(option.text)
except:
    print("Element non trouvé")


<selenium.webdriver.remote.webelement.WebElement (session="eaa6526b39e2105c23239c4b27a7fa56", element="f.9DE8707C9116CADA51E0D830377D59CF.d.671935E017091FDA8A1DDC827C0DE624.e.38")>


In [ ]:
clear_ant_select_choices()
